In [140]:
#!/usr/bin/env python
# coding: utf-8

from lib2to3.pytree import convert
import pandas
import numpy as np
from datetime import datetime
import os

#from tqdm import tqdm

os.chdir("/Users/pbenson/Documents/IOH project/")

txn_dirPath = 'Data/'


txn_converter = {'TXN - Transaction Type': str,
                 'TXN - Transaction Date': str,
                 'TXN - Item ID': str,
                 'TXN - Qty': float,
                 'TXN - Total Cost': float}

transaction_col = ['TXN - Transaction Type', 'TXN - Transaction Date',
                   'TXN - Item ID', 'TXN - Qty', 'TXN - Total Cost', 'TXN - Adjust Type']

out_types = ('051', '054', '030', '031')
positive_types = (41, 22, 24, 50, 10, 20)
drop_types = ('060', '053', '052', '012', '001', '042')

#date = '2021-12-31 23:59:59'

def get_lof_csv(directory):
    lof = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            lof.append(directory + file)
            print("Read file {}".format(file))
    return lof

def read_csv_to_df(lof, converter):
    print("Converting to pandas dataframe")

    dataframe = pandas.DataFrame()

    # More efficient way to do this: pd.concat
    next_txns = []
    for f in lof:
        next_txn = pandas.read_csv(f, converters=converter, encoding= 'unicode_escape')
        next_txns.append(next_txn)
        #dataframe = dataframe.append(next_txn, ignore_index=True)
        print(f + ' is appened')
    dataframe = pandas.concat(next_txns, ignore_index=True)
    return dataframe

def handle_I_D(txn_df):
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    
    
    for t_type in out_types: 
        txn_df.loc[txn_df['TXN - Transaction Type'] == t_type, 'TXN - Qty'] = 0 - abs(txn_df['TXN - Qty'])
        txn_df.loc[txn_df['TXN - Transaction Type'] == t_type, 'TXN - Total Cost'] = 0 - abs(txn_df['TXN - Total Cost'])

    for t_type in drop_types:
        txn_df.loc[txn_df['TXN - Transaction Type'] == t_type, 'TXN - Qty'] = 0 
        txn_df.loc[txn_df['TXN - Transaction Type'] == t_type, 'TXN - Total Cost'] = 0
    
    # Trust Anni's Code – it has already been validated.

    # We don't want to drop columns.
    #txn_df = txn_df.drop(columns=['TXN - Transaction Type', 'TXN - Adjust Type'])
    return txn_df

def read_txn_by_date(txn_df, date):
    txn_df = txn_df.loc[txn_df['TXN - Transaction Date'] < date]
    #txn_df = txn_df.drop(columns=['TXN - Transaction Date'])
    # We don't want to drop the Transaction Date column.
    return txn_df

def sum_by_date(txn_df, date):
    txn_df = txn_df.loc[txn_df['TXN - Transaction Date'] < date]
    #txn_df = txn_df.drop(columns=['TXN - Transaction Date'])
    # We don't want to drop the Transaction Date column.
    return txn_df["TXN - Total Cost"].sum()

def sku_running_total(txn_df: pandas.DataFrame, item_sku: str) -> float:
    return txn_df.loc[txn_df['TXN - Item ID'] == item_sku, ('TXN - Qty', 'TXN - Transaction Type')]

def thousand_places(number):
    # Simple function for formatting large number outputs
    number = round(number, 2)
    return "{:,}".format(number)

def validate_skus(txn_df):
    # Process txn_df into sku_sum dataframe
    sku_sums = txn_df.groupby(['TXN - Item ID'])['TXN - Qty'].sum()
    
    # Read in and process valid SKU's
    valid_sku_sums_df = pandas.read_csv('Data/Valid_End_Quantities.csv')
    
    valid_sku_sums_df.columns = valid_sku_sums_df.iloc[0] 
    valid_sku_sums_df = valid_sku_sums_df[1:]
    
    valid_sku_sums_df["IOH - Qty On Hand"] = pandas.to_numeric(valid_sku_sums_df["IOH - Qty On Hand"])
    valid_sku_sums_df.rename(columns={'IOH - Item ID':'TXN - Item ID'}, inplace=True)
    
    valid_sku_sums = valid_sku_sums_df.groupby(['TXN - Item ID'])['IOH - Qty On Hand'].sum().to_frame()
    
    
    # Merge SKU sums dataframes
    merged_sku_sums = valid_sku_sums.join(other=sku_sums, on='TXN - Item ID')
    merged_sku_sums.reset_index(inplace=True)
    merged_sku_sums.rename(columns={'index':'TXN - Item ID'}, inplace=True)
    
    merged_sku_sums['TXN - Qty'].fillna(0, inplace=True)
    
    # Calculate difference
    merged_sku_sums['Qty Difference'] = merged_sku_sums['IOH - Qty On Hand'] - merged_sku_sums['TXN - Qty']
    
    return merged_sku_sums

def get_valid_percent(merged_skus):
    return (len(merged_skus) - len(merged_skus.loc[merged_skus['Qty Difference'] != 0])) / len(merged_skus)


In [2]:
txn_df = read_csv_to_df(get_lof_csv(txn_dirPath + "CSV_Files/"), txn_converter)


dataframe = txn_df.copy(deep = True) 

# # drop duplication
dataframe = dataframe.drop_duplicates()
# # filter dataframe
dataframe = dataframe.loc[dataframe['TXN - Transaction Type'].isin(['010', '020', '022', '024', '030', '031', '041', '050', '051', '054'])]
#dataframe
"""

full_dataframe_path = "Data/full_dataframe.tsv"

# Tab-separated is a better bet than csv.
print("Reading Dataframe")
dataframe = pandas.read_csv(full_dataframe_path, converters=txn_converter, sep="\t")

print(dataframe.head(10))
"""

# NOTE: this program seems to work fine when building the dataframe from scratch (using the csv files). 
# However, it fails when trying to build from the exported version of the same dataframe (full_dataframe.tsv). 
# I suspect there is slight deviation during export/import

#print("Filtering by date")
#dataframe = read_txn_by_date(dataframe,date)
#print(dataframe.loc[dataframe['TXN - Transaction Type'] == '031', 'TXN - Qty'])
print("Handling ID")

dataframe = handle_I_D(dataframe)
#print(dataframe.loc[dataframe['TXN - Transaction Type'] == '031', 'TXN - Qty'])
#dataframe.loc[dataframe['TXN - Transaction Type'] == '031', 'TXN - Qty'] = 0 - abs(dataframe['TXN - Qty'])
#print(dataframe.loc[dataframe['TXN - Transaction Type'] == '031', 'TXN - Qty'])


print("Balance for 00250052:")
print("Sum:", sku_running_total(dataframe, "00250052")['TXN - Qty'].sum())

print("Balance for 42082077:", sku_running_total(dataframe, "42082077")['TXN - Qty'].sum())

print("Balance for 00250066:", sku_running_total(dataframe, "00250066")['TXN - Qty'].sum())


# Process: 
# - load all csv's 
# - Deep copy/drop duplicates
# - Run new handle_I_D function, don't filter by date
# - Run sku_running_total\['TXN-Qty'\].sum()

### Dump SKU Quantities to csv

# TODO for 1/28/22: Get end quantities for every SKU.

sku_counts = dataframe.groupby(['TXN - Item ID'])['TXN - Qty'].sum()

print("Validating SKU Counts")
print(get_valid_percent(validate_skus(dataframe)))

print("Dumping SKU counts to dataframe")
#sku_counts.to_csv("Data/unique_sku_counts.tsv", sep = "\t")

Read file PL_INVENTORY_TRANSACTIONS_2009_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2019_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2020_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2006_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2016_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2012_Q3x.csv
Read file PL_INVENTORY_TRANSACTIONS_2013_Q2x.csv
Read file PL_INVENTORY_TRANSACTIONS_2010_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2017_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2007_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2013_Q3x.csv
Read file PL_INVENTORY_TRANSACTIONS_2012_Q2x.csv
Read file PL_INVENTORY_TRANSACTIONS_2011_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2018_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2008_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2011_Q3x.csv
Read file PL_INVENTORY_TRANSACTIONS_2005_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2015_Q1x.csv
Read file PL_INVENTORY_TRANSACTIONS_2013_Q4x.csv
Read file PL_INVENTORY_TRANSACTIONS_2010_Q2x.csv
Read file PL_INVENTO

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/445560429.py:1: DtypeWarning: Columns (33,40,50,54,59) have mixed types.Specify dtype option on import or set low_memory=False.
  txn_df = read_csv_to_df(get_lof_csv(txn_dirPath + "CSV_Files/"), txn_converter)


Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2009_Q4x.csv is appened


/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/445560429.py:1: DtypeWarning: Columns (19,33,36,40,59) have mixed types.Specify dtype option on import or set low_memory=False.
  txn_df = read_csv_to_df(get_lof_csv(txn_dirPath + "CSV_Files/"), txn_converter)


Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2019_Q4x.csv is appened


...

Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2013_Q1x.csv is appened
Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2007_Q3x.csv is appened


/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/445560429.py:1: DtypeWarning: Columns (19,36,40,44,59,66) have mixed types.Specify dtype option on import or set low_memory=False.
  txn_df = read_csv_to_df(get_lof_csv(txn_dirPath + "CSV_Files/"), txn_converter)


Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2017_Q3x.csv is appened


/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/445560429.py:1: DtypeWarning: Columns (19,33,36,40,44,59) have mixed types.Specify dtype option on import or set low_memory=False.
  txn_df = read_csv_to_df(get_lof_csv(txn_dirPath + "CSV_Files/"), txn_converter)


Data/CSV_Files/PL_INVENTORY_TRANSACTIONS_2020_Q2x.csv is appened
Handling ID
Balance for 00250052:
Sum: 2630.0
Balance for 42082077: 71.0
Balance for 00250066: 2146.0
Validating SKU Counts
0.9880386443796964
Dumping SKU counts to dataframe


In [4]:
dataframe['Updated_Total_Cost'] = dataframe['TXN - Qty'] * dataframe['TXN - Avg Matl Cost']

In [8]:
dataframe.groupby('TXN - Item ID')['TXN - Qty'].sum().min()

-619.0

In [14]:
print('Total Value of Inventory On Hand:', thousand_places(sum(dataframe.groupby('TXN - Item ID')['Updated_Total_Cost'].sum())))

Total Value of Inventory On Hand: 65,871,372.24


In [17]:
# This script uses the old Total Cost
print('Total Value of Inventory On Hand:', thousand_places(sum(dataframe.groupby('TXN - Item ID')['TXN - Total Cost'].sum())))

Total Value of Inventory On Hand: 65,881,211.46


In [15]:
def total_ioh_value(txn_df):
    txn_df = handle_I_D(txn_df)
    
    txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']
    return sum(txn_df.groupby('TXN - Item ID')['Updated_Total_Cost'].sum())

Now to get this by date. This should be a simple filtering operation. 

In [27]:
ex_date = dataframe['TXN - Transaction Date'][0]

In [33]:
ex_date_date = datetime.strptime(ex_date, '%Y/%m/%d')

In [31]:
print(ex_date)
print(type(ex_date_date))

2009/10/01
<class 'datetime.datetime'>


In [38]:
bad_date = '1/10/2009'
bad_date_formatted = datetime.strptime(bad_date, '%d/%m/%Y')
print(bad_date_formatted.date())

2009-10-01


In [47]:
def get_ioh_by_date(txn_df, filter_date):
    # Assume date is in format 'm d Y'
    filter_date_formatted = datetime.strptime(filter_date, '%m/%d/%Y')
    
    # Filter by date. Convert txn_df['TXN - Transaction Date'] and 'filter_date' to a usable date format
    txn_df['TXN - Transaction Date'] = pandas.to_datetime(txn_df['TXN - Transaction Date'])
    
    txn_df_filtered = txn_df.loc[txn_df['TXN - Transaction Date'] < filter_date_formatted]
    
    # Call total_ioh_value
    
    return total_ioh_value(txn_df_filtered)

In [66]:
get_ioh_by_date(dataframe, '12/31/2020')

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


66308140.855925396

TODO: Implement a caching system to speed up IOH by date.

## Cached Dates

In [119]:
# txn_df is the dataframe, date_list is a list of strings in m/d/Y date format
def get_ioh_multiple_dates(txn_df, date_list):
    ioh_by_day = {}
    
    for i in range(len(date_list)):
        date_list[i] = datetime.strptime(date_list[i], '%m/%d/%Y')
        #print(type(date))
    # Sort date_list
    start_date = datetime.strptime('1/1/2004', '%m/%d/%Y')
    
    date_list.sort()
    
    # Convert dataframe date column to datetime.
    txn_df['TXN - Transaction Date'] = pandas.to_datetime(txn_df['TXN - Transaction Date'])
    
    # Calculate IOH for the first date. This will become the basis for the subsequent calculations.
    txn_df_filtered = txn_df.loc[txn_df['TXN - Transaction Date'] < date_list[0]]
    
    # Call total_ioh_value
    ioh_by_day[date_list[0].strftime('%m/%d/%Y')] = total_ioh_value(txn_df_filtered)
    
    # Iterate through the rest of the dates
    for i in range(1, len(date_list)):
        #print(date_list[i], date_list[i-1])
        filtered_df = txn_df.loc[(txn_df['TXN - Transaction Date'] < date_list[i]) & (txn_df['TXN - Transaction Date'] >= date_list[i-1])]
        
        ioh_by_day[date_list[i].strftime('%m/%d/%Y')] = ioh_by_day[date_list[i-1].strftime('%m/%d/%Y')] + total_ioh_value(filtered_df)
    
    return ioh_by_day

In [115]:
print(get_ioh_multiple_dates(dataframe, ['1/1/2005', '1/1/2006', '1/1/2009']))

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


2006-01-01 00:00:00 2005-01-01 00:00:00
2009-01-01 00:00:00 2006-01-01 00:00:00
{'01/01/2005': 51601952.5013997, '01/01/2006': 52600313.7322997, '01/01/2009': 62402307.25749953}


In [116]:
print(get_ioh_multiple_dates(dataframe, ['1/1/2005', '10/1/2006']))

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


2006-10-01 00:00:00 2005-01-01 00:00:00
{'01/01/2005': 51601952.5013997, '10/01/2006': 56253690.23229969}


In [117]:
print(get_ioh_multiple_dates(dataframe, ['1/1/2005', '1/1/2006', '1/1/2009', '3/1/2020', '1/1/2021']))

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


2006-01-01 00:00:00 2005-01-01 00:00:00
2009-01-01 00:00:00 2006-01-01 00:00:00
2020-03-01 00:00:00 2009-01-01 00:00:00
2021-01-01 00:00:00 2020-03-01 00:00:00
{'01/01/2005': 51601952.5013997, '01/01/2006': 52600313.7322997, '01/01/2009': 62402307.25749953, '03/01/2020': 69248771.94670324, '01/01/2021': 66471469.685715236}


In [93]:
print(get_ioh_by_date(dataframe, '3/1/2020'))

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


69248771.94670354


In [86]:
print(get_ioh_by_date(dataframe, '1/1/2021'))

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


66471469.685715325


David wants a list of the numbers by the end of the month since 2004 (Use 1st of every month because it's exclusive). Dump this to txt file.

In [136]:
months_since_2004 = []

for i in range(2004, 2021):
    for j in range(1, 13):
        months_since_2004.append("{}/1/{}".format(j, i))

months_since_2004.append('1/1/2021')
    
all_months_dict = get_ioh_multiple_dates(dataframe, months_since_2004)

months_df = pandas.DataFrame.from_dict({'date': all_months_dict.keys(), 'Inventory On Hand value': all_months_dict.values()})

/var/folders/73/z_zg2ww102j_zl0j1rp57xxcwgxbxw/T/ipykernel_41865/2815660484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_df['Updated_Total_Cost'] = txn_df['TXN - Qty'] * txn_df['TXN - Avg Matl Cost']


In [137]:
months_df.loc[months_df['date'] == '03/01/2020']

,date,Inventory On Hand value
194,03/01/2020,6.924877e+07


In [138]:
months_df = months_df.round(decimals=3)
list(months_df['Inventory On Hand value'])

[0.0,
 0.0,
 0.0,
 0.0,
 44497395.648,
 44218436.629,
 43674590.13,
 43634260.945,
 44606822.211,
 44653207.588,
 45823902.181,
 50625216.698,
 51601952.501,
 47888841.688,
 48817904.839,
 50043983.091,
 50246813.429,
 50983549.543,
 51652321.213,
 50981645.151,
 51882555.173,
 52348325.406,
 52717791.18,
 53560209.513,
 52600313.732,
 54291354.663,
 54190892.877,
 54460555.557,
 55075677.864,
 55191766.249,
 55085270.82,
 55018767.264,
 55659741.022,
 56253690.232,
 56548545.858,
 56990752.92,
 57313874.919,
 58719939.051,
 59954769.885,
 60800265.648,
 60848889.693,
 59877924.888,
 61649302.87,
 61670314.386,
 62362704.324,
 64154109.688,
 63569107.131,
 63740193.244,
 63572930.487,
 63211785.522,
 62439093.102,
 62940078.856,
 62287800.278,
 62033136.731,
 62828405.926,
 62555696.901,
 62753340.996,
 61542992.691,
 61578310.34,
 61849214.827,
 62402307.257,
 62626662.407,
 62495653.898,
 62402942.687,
 62431691.137,
 62570658.969,
 64057899.862,
 63100395.804,
 64064458.727,
 633626

In [139]:
months_df.to_csv('/Users/pbenson/Documents/IOH project/IOH_by_month.csv')